##### This notebook collects all the survey data from MongoDB and creates the corresponding dataframe.

In [96]:
import os
import warnings
import datetime
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from functions import data_loading
from functions import surveys_scoring
from dataprep.eda import create_report

warnings.filterwarnings("ignore")

Connect securely to the database

In [97]:
load_dotenv("../config.env")
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
client = MongoClient("mongodb://" + MONGO_USER + ":" + MONGO_PASSWORD + "@localhost:27017/")
db = client.rais

#### Survey data

Find all users provided their survey data

In [98]:
users = db.surveys.distinct('user_id')
print(len(users), "users provided their survey data")

67 users provided their survey data


Find all the data types

In [99]:
types = db.surveys.distinct('type')
types

['bfpt', 'breq', 'dq', 'panas', 'stai', 'ttmspbf']

Load fitbit data

In [100]:
fitbit_df = pd.read_pickle('../data/temp_fitbit.pkl')
fitbit_df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,very_active_minutes,minutes_below_zone_1,minutes_in_zone_1,minutes_in_zone_2,minutes_in_zone_3,water_amount,mood_value,gender,age,bmi
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,0.0,1303.0,119.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,0.0,1400.0,36.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164490,621e36bb67b776a240b40d64,2022-01-18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164491,621e36bb67b776a240b40d64,2022-01-19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164492,621e36bb67b776a240b40d64,2022-01-20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0
164493,621e36bb67b776a240b40d64,2022-01-21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,24.0


##### bfpt

In [101]:
bfpt = pd.DataFrame(columns=["user_id", "data"])

# read and load from MongoDB
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "bfpt"},
            {"user_id": user}
        ]},{"_id": 0})))
    bfpt = pd.concat([bfpt, user_data], axis=0)

# split data column (json format) into two columns (df format)
for ipip in range(1,51):
    if ipip < 10:
        col_name = "ipip[SQ00{}]".format(ipip)
    else:
        col_name = "ipip[SQ0{}]".format(ipip)
    bfpt[col_name] = bfpt["data"].apply(lambda d: d[col_name])
bfpt.drop(['data', 'type'], axis=1, inplace=True)

# categorization of answers into positive and negative ones
bfpt_scoring = pd.read_csv("../data/BFPT-Coding.csv", sep=";")
ipip_plus = bfpt_scoring[bfpt_scoring.plus == True].code
ipip_minus = bfpt_scoring[bfpt_scoring.plus == False].code

# Based on the scoring document: https://openpsychometrics.org/printable/big-five-personality-test.pdf, some questions are reversed
for col in bfpt.columns:
    if col in ipip_minus.values:
        bfpt[col] = bfpt[col].apply(lambda score: surveys_scoring.inverse_score(score, 1, 5))

# calculation of the score of each personality type
ipip_extraversion = bfpt_scoring[bfpt_scoring.factor == 1].code
ipip_agreeableness = bfpt_scoring[bfpt_scoring.factor == 2].code
ipip_conscientiousness = bfpt_scoring[bfpt_scoring.factor == 3].code
ipip_stability = bfpt_scoring[bfpt_scoring.factor == 4].code
ipip_intellect = bfpt_scoring[bfpt_scoring.factor == 5].code
bfpt["extraversion"] = bfpt[ipip_extraversion].sum(axis=1)
bfpt["agreeableness"] = bfpt[ipip_agreeableness].sum(axis=1)
bfpt["conscientiousness"] = bfpt[ipip_conscientiousness].sum(axis=1)
bfpt["stability"] = bfpt[ipip_stability].sum(axis=1)
bfpt["intellect"] = bfpt[ipip_intellect].sum(axis=1)

# drop unnecessary columns and rename columns
bfpt = bfpt[bfpt.columns.drop(list(bfpt.filter(regex='ipip\[SQ')))]
bfpt.rename(columns={'user_id': 'id'}, inplace=True)

# convert the score into category using mean and std of a specific subgroup (in this case: gender)
# add the gender information for each participant
bfpt = bfpt.merge(fitbit_df[['id','gender']].drop_duplicates(), how='left')
# there are two missing values
bfpt.gender.fillna('FEMALE', inplace=True)
# calculate means and stds
MEAN_1_FEMALE, STD_1_FEMALE = bfpt.groupby('gender').extraversion.mean()["FEMALE"], bfpt.groupby('gender').extraversion.std()["FEMALE"]
MEAN_1_MALE, STD_1_MALE = bfpt.groupby('gender').extraversion.mean()["MALE"], bfpt.groupby('gender').extraversion.std()["MALE"]
MEAN_2_FEMALE, STD_2_FEMALE = bfpt.groupby('gender').agreeableness.mean()["FEMALE"], bfpt.groupby('gender').agreeableness.std()["FEMALE"]
MEAN_2_MALE, STD_2_MALE = bfpt.groupby('gender').agreeableness.mean()["MALE"], bfpt.groupby('gender').agreeableness.std()["MALE"]
MEAN_3_FEMALE, STD_3_FEMALE = bfpt.groupby('gender').conscientiousness.mean()["FEMALE"], bfpt.groupby('gender').conscientiousness.std()["FEMALE"]
MEAN_3_MALE, STD_3_MALE = bfpt.groupby('gender').conscientiousness.mean()["MALE"], bfpt.groupby('gender').conscientiousness.std()["MALE"]
MEAN_4_FEMALE, STD_4_FEMALE = bfpt.groupby('gender').stability.mean()["FEMALE"], bfpt.groupby('gender').stability.std()["FEMALE"]
MEAN_4_MALE, STD_4_MALE = bfpt.groupby('gender').stability.mean()["MALE"], bfpt.groupby('gender').stability.std()["MALE"]
MEAN_5_FEMALE, STD_5_FEMALE = bfpt.groupby('gender').intellect.mean()["FEMALE"], bfpt.groupby('gender').intellect.std()["FEMALE"]
MEAN_5_MALE, STD_5_MALE = bfpt.groupby('gender').intellect.mean()["MALE"], bfpt.groupby('gender').intellect.std()["MALE"]
# convert the score into category
bfpt['extraversion'] = bfpt.apply(lambda row: surveys_scoring.personality_category(row.extraversion, MEAN_1_MALE, STD_1_MALE) if row.gender == "MALE" else surveys_scoring.personality_category(row.extraversion, MEAN_1_FEMALE, STD_1_FEMALE), axis=1)
bfpt['agreeableness'] = bfpt.apply(lambda row: surveys_scoring.personality_category(row.agreeableness, MEAN_2_MALE, STD_2_MALE) if row.gender == "MALE" else surveys_scoring.personality_category(row.agreeableness, MEAN_2_FEMALE, STD_2_FEMALE), axis=1)
bfpt['conscientiousness'] = bfpt.apply(lambda row: surveys_scoring.personality_category(row.conscientiousness, MEAN_3_MALE, STD_3_MALE) if row.gender == "MALE" else surveys_scoring.personality_category(row.conscientiousness, MEAN_3_FEMALE, STD_3_FEMALE), axis=1)
bfpt['stability'] = bfpt.apply(lambda row: surveys_scoring.personality_category(row.stability, MEAN_4_MALE, STD_4_MALE) if row.gender == "MALE" else surveys_scoring.personality_category(row.stability, MEAN_4_FEMALE, STD_4_FEMALE), axis=1)
bfpt['intellect'] = bfpt.apply(lambda row: surveys_scoring.personality_category(row.intellect, MEAN_5_MALE, STD_5_MALE) if row.gender == "MALE" else surveys_scoring.personality_category(row.intellect, MEAN_5_FEMALE, STD_5_FEMALE), axis=1)

# drop unnecessary columns
bfpt.drop(columns=['gender'], inplace=True)

bfpt

,id,extraversion,agreeableness,conscientiousness,stability,intellect
0,621e2e8e67b776a24055b564,LOW,LOW,HIGH,HIGH,HIGH
1,621e2eaf67b776a2406b14ac,AVERAGE,HIGH,AVERAGE,LOW,HIGH
2,621e2ed667b776a24085d8d1,HIGH,AVERAGE,LOW,AVERAGE,AVERAGE
3,621e2f3967b776a240c654db,AVERAGE,AVERAGE,LOW,HIGH,AVERAGE
4,621e2f6167b776a240e082a9,HIGH,AVERAGE,AVERAGE,HIGH,LOW
5,621e2f7a67b776a240f14425,AVERAGE,AVERAGE,HIGH,HIGH,AVERAGE
6,621e2f9167b776a240011ccb,LOW,HIGH,AVERAGE,LOW,HIGH
7,621e2fb367b776a24015accd,HIGH,AVERAGE,LOW,LOW,HIGH
8,621e2fce67b776a240279baa,HIGH,HIGH,AVERAGE,LOW,AVERAGE
9,621e2ff067b776a2403eb737,AVERAGE,HIGH,LOW,LOW,LOW


In [102]:
# merge with fitbit data
df = fitbit_df.merge(bfpt, how='outer', on='id')
df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,water_amount,mood_value,gender,age,bmi,extraversion,agreeableness,conscientiousness,stability,intellect
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164490,621e36bb67b776a240b40d64,2022-01-18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,24.0,HIGH,LOW,AVERAGE,LOW,LOW
164491,621e36bb67b776a240b40d64,2022-01-19,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,24.0,HIGH,LOW,AVERAGE,LOW,LOW
164492,621e36bb67b776a240b40d64,2022-01-20,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,24.0,HIGH,LOW,AVERAGE,LOW,LOW
164493,621e36bb67b776a240b40d64,2022-01-21,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,FEMALE,<30,24.0,HIGH,LOW,AVERAGE,LOW,LOW


##### breq

In [103]:
breq = pd.DataFrame(columns=["user_id", "data"])

# read and load from MongoDB
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "breq"},
            {"user_id": user}
        ]},{"_id": 0})))
    breq = pd.concat([breq, user_data], axis=0)

# process the datetime object
breq["date"] = breq["data"].apply(lambda d: d["submitdate"])
breq = data_loading.date_conversion(breq)

# split data column (json format) into two columns (df format)
for engage in range(1,20):
    if engage < 10:
        col_name = "engage[SQ00{}]".format(engage)
    else:
        col_name = "engage[SQ0{}]".format(engage)
    breq[col_name] = breq["data"].apply(lambda d: d[col_name])
breq.drop(['data', 'type'], axis=1, inplace=True)

# categorization and scoring of participants' regulations based on breq-2's scoring document
breq["amotivation"] = breq[["engage[SQ005]", "engage[SQ009]", "engage[SQ012]", "engage[SQ019]"]].mean(axis=1)
breq["external_regulation"] = breq[["engage[SQ001]", "engage[SQ006]", "engage[SQ011]", "engage[SQ016]"]].mean(axis=1)
breq["introjected_regulation"] = breq[["engage[SQ002]", "engage[SQ007]", "engage[SQ013]"]].mean(axis=1)
breq["identified_regulation"] = breq[["engage[SQ003]", "engage[SQ008]", "engage[SQ014]", "engage[SQ017]"]].mean(axis=1)
breq["intrinsic_regulation"] = breq[["engage[SQ004]", "engage[SQ010]", "engage[SQ015]", "engage[SQ018]"]].mean(axis=1)

# identify participants' final regulation based on the one collected the maximum score
temp = breq[["amotivation", "external_regulation", "introjected_regulation", "identified_regulation", "intrinsic_regulation"]].agg(['idxmax','max'], axis=1).mask(lambda x: x['max'].eq(0))
breq.loc[:, "self_determination"] = temp.loc[:, "idxmax"]

# drop unnecessary columns and rename columns
breq.rename(columns={'user_id': 'id'}, inplace=True)
breq = breq[['id', 'date', 'hour', 'self_determination']]

# check for duplicates
# Findings: 3 duplicates found based on (id, date) which is wrong, since this answer had to be collected max once at the beginning of the experiment and max once at the end of the experiment
# Approach: keep the latest record
breq.sort_values(by=['id', 'date', 'hour'], inplace=True)
breq.drop_duplicates(subset=["id", "date"], inplace=True, keep="last")

breq

,id,date,hour,self_determination
0,621e2e8e67b776a24055b564,2021-05-31,13,intrinsic_regulation
1,621e2e8e67b776a24055b564,2021-07-26,13,intrinsic_regulation
0,621e2eaf67b776a2406b14ac,2021-11-29,11,identified_regulation
1,621e2eaf67b776a2406b14ac,2022-01-17,12,identified_regulation
0,621e2ed667b776a24085d8d1,2021-07-06,17,introjected_regulation
...,...,...,...,...
0,621e36c267b776a240ba2756,2021-05-31,13,intrinsic_regulation
0,621e36f967b776a240e5e7c9,2021-05-31,12,introjected_regulation
1,621e36f967b776a240e5e7c9,2021-07-26,12,introjected_regulation
0,621e375b67b776a240290cdc,2021-01-06,15,identified_regulation


In [104]:
# merge with fitbit data
df = df.merge(breq, how='outer', on=['id', 'date', 'hour'])
df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,mood_value,gender,age,bmi,extraversion,agreeableness,conscientiousness,stability,intellect,self_determination
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,FEMALE,<30,>=25,AVERAGE,HIGH,LOW,LOW,LOW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164518,621e362467b776a2404ad513,2021-01-08,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intrinsic_regulation
164519,621e362467b776a2404ad513,2021-02-06,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,identified_regulation
164520,621e367e67b776a24087d75d,2022-01-27,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,amotivation
164521,621e36f967b776a240e5e7c9,2021-07-26,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,introjected_regulation


##### panas

In [105]:
panas = pd.DataFrame(columns=["user_id", "data"])

# read and load from MongoDB
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "panas"},
            {"user_id": user}
        ]},{"_id": 0})))
    panas = pd.concat([panas, user_data], axis=0)

# process the datetime object
panas["date"] = panas["data"].apply(lambda d: d["submitdate"])
panas = data_loading.date_conversion(panas)

# split data column (json format) into two columns (df format)
for p in range(1,21):
    if p < 10:
        col_name = "P1[SQ00{}]".format(p)
    else:
        col_name = "P1[SQ0{}]".format(p)
    panas[col_name] = panas["data"].apply(lambda d: d[col_name])
panas.drop(['data', 'type'], axis=1, inplace=True)

# calculation of the score of negative and positive affect schedule
positive = ["P1[SQ001]", "P1[SQ003]", "P1[SQ005]", "P1[SQ009]", "P1[SQ010]", "P1[SQ012]", "P1[SQ014]", "P1[SQ016]", "P1[SQ017]", "P1[SQ019]"]
negative = ["P1[SQ002]", "P1[SQ004]", "P1[SQ006]", "P1[SQ007]", "P1[SQ008]", "P1[SQ011]", "P1[SQ013]", "P1[SQ015]", "P1[SQ018]", "P1[SQ020]"]
panas["positive_affect_score"] = panas[positive].sum(axis=1)
panas["negative_affect_score"] = panas[negative].sum(axis=1)

# convert the score into stress category using mean and std of a specific subgroup
negative_mean_panas = panas['negative_affect_score'].mean()
negative_std_panas = panas['negative_affect_score'].std()
panas['negative_affect_score'] = panas['negative_affect_score'].apply(lambda score: surveys_scoring.get_stai_category(score, negative_mean_panas, negative_std_panas))
positive_mean_panas = panas['positive_affect_score'].mean()
positive_std_panas = panas['positive_affect_score'].std()
panas['positive_affect_score'] = panas['positive_affect_score'].apply(lambda score: surveys_scoring.get_stai_category(score, positive_mean_panas, positive_std_panas))

# drop unnecessary columns and rename columns
panas = panas[panas.columns.drop(list(panas.filter(regex='P1\[SQ')))]
panas.rename(columns={'user_id': 'id'}, inplace=True)

# check for duplicates
# Findings: 24 duplicates found based on (id, date) which is wrong, since this answer had to be collected max once per two weeks
# Approach: keep the latest record
panas.sort_values(by=['id', 'date', 'hour'], inplace=True)
panas.drop_duplicates(subset=["id", "date"], inplace=True, keep="last")

panas

,id,date,hour,positive_affect_score,negative_affect_score
2,621e2e8e67b776a24055b564,2021-05-31,13,Above average,Below average
4,621e2e8e67b776a24055b564,2021-06-28,17,Average,Below average
3,621e2e8e67b776a24055b564,2021-07-06,12,Above average,Below average
5,621e2e8e67b776a24055b564,2021-07-19,12,Above average,Below average
1,621e2e8e67b776a24055b564,2021-07-26,12,Above average,Below average
...,...,...,...,...,...
1,621e375367b776a24021e950,2022-02-01,17,Average,Above average
0,621e375b67b776a240290cdc,2021-01-06,14,Below average,Above average
3,621e375b67b776a240290cdc,2021-06-07,11,Average,Above average
2,621e375b67b776a240290cdc,2021-06-22,11,Average,Above average


In [106]:
# merge with fitbit data
df = fitbit_df.merge(panas, how='outer', on=['id', 'date', 'hour'])
df

,id,date,hour,ecg,heart_rate_alert,type,nightly_temperature,nremhr,spo2,rmssd,...,minutes_in_zone_1,minutes_in_zone_2,minutes_in_zone_3,water_amount,mood_value,gender,age,bmi,positive_affect_score,negative_affect_score
0,621e2ff067b776a2403eb737,2021-12-22,19,NSR,NONE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25,NaN,NaN
1,621e2ff067b776a2403eb737,2021-11-18,0,NaN,NaN,SKIN,35.025730,NaN,NaN,NaN,...,119.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25,NaN,NaN
2,621e2ff067b776a2403eb737,2021-11-18,21,NaN,NaN,SKIN,34.866951,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25,NaN,NaN
3,621e2ff067b776a2403eb737,2021-11-20,0,NaN,NaN,SKIN,35.349583,NaN,NaN,NaN,...,36.0,0.0,0.0,NaN,NaN,FEMALE,<30,>=25,NaN,NaN
4,621e2ff067b776a2403eb737,2021-11-20,23,NaN,NaN,SKIN,34.495486,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,FEMALE,<30,>=25,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164577,621e36f967b776a240e5e7c9,2021-05-07,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Above average,Below average
164578,621e36f967b776a240e5e7c9,2021-07-26,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Below average,Above average
164579,621e36f967b776a240e5e7c9,2021-12-07,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Average,Average
164580,621e375367b776a24021e950,2022-02-01,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Average,Above average


##### stai

In [107]:
# stai = pd.DataFrame(columns=["user_id", "data"])
#
# # read and load from MongoDB
# for user in users:
#     user_data = pd.DataFrame(list(
#         db.surveys.find({"$and": [
#             {"type": "stai"},
#             {"user_id": user}
#         ]},{"_id": 0})))
#     stai = pd.concat([stai, user_data], axis=0)
#
# # process the datetime object
# stai["date"] = stai["data"].apply(lambda d: d["submitdate"])
# stai = data_loading.date_conversion(stai)
#
# # split data column (json format) into two columns (df format)
# for engage in range(1,21):
#     if engage < 10:
#         col_name = "STAI[SQ00{}]".format(engage)
#     else:
#         col_name = "STAI[SQ0{}]".format(engage)
#     stai[col_name] = stai["data"].apply(lambda d: d[col_name])
# stai.drop(['data', 'type'], axis=1, inplace=True)
#
# # convert 5-likert to 4-likert scale (mistakenly the STAI scale was distributed with a 5-likert, but it's originally 4-likert)
# stai.iloc[:, 3:] = stai.iloc[:, 3:].apply(lambda x: surveys_scoring.convert_5_to_4_likert(x))
# for col in stai.iloc[:, 3:].columns:
#     stai[col] = stai[col].apply(lambda x: surveys_scoring.proper_round(x))
#
# # Based on the scoring document: https://oml.eular.org/sysModules/obxOML/docs/id_150/State-Trait-Anxiety-Inventory.pdf, some questions are reversed in STAI
# stai_reversed = ["STAI[SQ001]", "STAI[SQ002]", "STAI[SQ005]", "STAI[SQ008]", "STAI[SQ010]", "STAI[SQ011]", "STAI[SQ015]", "STAI[SQ016]", "STAI[SQ019]", "STAI[SQ020]"]
# for col in stai.columns:
#     if col in stai_reversed:
#         stai[col] = stai[col].apply(lambda score: surveys_scoring.inverse_score(score, 1, 4))
#
# # to calculate the total stress score simply sum per
# stai['stai_stress'] = stai.iloc[:, 3:].sum(axis=1)
#
# # convert the score into stress category using mean and std of a specific subgroup
# mean_stai = stai['stai_stress'].mean()
# std_stai = stai['stai_stress'].std()
# stai['stai_stress'] = stai['stai_stress'].apply(lambda score: surveys_scoring.get_stai_category(score, mean_stai, std_stai))
#
# # drop unnecessary columns and rename columns
# stai = stai[stai.columns.drop(list(stai.filter(regex='STAI\[SQ')))]
# stai.rename(columns={'user_id': 'id'}, inplace=True)
#
# # check for duplicates
# # Findings: 25 duplicates found based on (id, date) which is wrong, since this answer had to be collected max once per two weeks
# # Approach: keep the latest record
# stai.sort_values(by=['id', 'date', 'hour'], inplace=True)
# stai.drop_duplicates(subset=["id", "date"], inplace=True, keep="last")
# stai

In [108]:
# # merge with fitbit data
# df = fitbit_df.merge(stai, how='outer', on=['id', 'date', 'hour'])
# duplicate = df[df.duplicated(['id', 'date', 'hour'])]
# duplicate
# df

##### ttmspbf

In [109]:
# ttm = pd.DataFrame(columns=["user_id", "data"])
# for user in users:
#     user_data = pd.DataFrame(list(
#         db.surveys.find({"$and": [
#             {"type": "ttmspbf"},
#             {"user_id": user}
#         ]}, {"_id": 0})))
#     ttm = pd.concat([ttm, user_data], axis=0)
#
# ttm["date"] = ttm["data"].apply(lambda d: d["submitdate"])
# ttm["stage"] = ttm["data"].apply(lambda d: d["stage"])
# ttm = data_loading.date_conversion(ttm)
#
# for engage in range(2,32):
#     if engage < 10:
#         col_name = "processes[SQ00{}]".format(engage)
#         new_col_name = "processes[SQ00{}]".format(engage-1)
#     else:
#         col_name = "processes[SQ0{}]".format(engage)
#         if engage == 10:
#             new_col_name = "processes[SQ00{}]".format(engage-1)
#         else:
#             new_col_name = "processes[SQ0{}]".format(engage-1)
#     ttm[new_col_name] = ttm["data"].apply(lambda d: d[col_name])
# ttm.drop(['data'], axis=1, inplace=True)
#
# ttm["stage"] = ttm.stage.apply(lambda response: surveys_scoring.define_stage_of_change(response))
# ttm
# # users_ttm.drop_duplicates(subset=["user_id", "submitdate"], keep="last", inplace=True)
# # print("We have {} survey responses for the TTM and Processes of Change scale after duplicate elimination.".format(users_ttm.shape[0]))
#
# # users_ttm["ttm_consciousness_raising"] = users_ttm[["processes[SQ001]","processes[SQ011]", "processes[SQ021]"]].mean(axis=1)
# # users_ttm["ttm_dramatic_relief"] = users_ttm[["processes[SQ002]","processes[SQ012]", "processes[SQ022]"]].mean(axis=1)
# # users_ttm["ttm_environmental_reevaluation"] = users_ttm[["processes[SQ003]","processes[SQ013]", "processes[SQ023]"]].mean(axis=1)
# # users_ttm["ttm_self_reevaluation"] = users_ttm[["processes[SQ004]","processes[SQ014]", "processes[SQ024]"]].mean(axis=1)
# # users_ttm["ttm_social_liberation"] = users_ttm[["processes[SQ005]","processes[SQ015]", "processes[SQ025]"]].mean(axis=1)
# # users_ttm["ttm_counterconditioning"] = users_ttm[["processes[SQ006]","processes[SQ016]", "processes[SQ026]"]].mean(axis=1)
# # users_ttm["ttm_helping_relationships"] = users_ttm[["processes[SQ007]","processes[SQ017]", "processes[SQ027]"]].mean(axis=1)
# # users_ttm["ttm_reinforcement_management"] = users_ttm[["processes[SQ008]","processes[SQ018]", "processes[SQ028]"]].mean(axis=1)
# # users_ttm["ttm_self_liberation"] = users_ttm[["processes[SQ009]","processes[SQ019]", "processes[SQ029]"]].mean(axis=1)
# # users_ttm["ttm_stimulus_control"] = users_ttm[["processes[SQ010]","processes[SQ020]", "processes[SQ030]"]].mean(axis=1)
#
# # users_ttm = users_ttm[users_ttm.columns.drop(list(users_ttm.filter(regex='processes')))]
